In [82]:
import sys
import os
import inspect
import numpy as np
import pandas as pd
import plotly.express as px
import json
import glob
import plotly.graph_objects as go
import json

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))))

from chemprop.train.uncertainty_evaluator import UncertaintyEvaluator

from plotly import io as pio

# Update Styling
pio.templates.default = 'ggplot2'

# Initial Calibration and Evaluation

### Setup

In [83]:
evaluation_metrics = ['spearman', 'log_likelihood', 'calibration_auc']

data_sources = ['delaney', 'freesolv', 'lipo', 'qm7', 'logp']

estimator_types = ['mpnn_ensemble', 'mpnn_bootstrap', 'mpnn_snapshot', 'mpnn_dropout10', 'mpnn_dropout20', 'mpnn_mve',
                   'mpnn_gaussian', 'mpnn_random_forest', 'mpnn_latent_space', 'mpnn_tanimoto',
                   'ffn_ensemble', 'ffn_bootstrap', 'ffn_snapshot', 'ffn_dropout10', 'ffn_dropout20', 'ffn_mve',
                   'ffn_gaussian', 'ffn_random_forest', 'ffn_latent_space', 'ffn_tanimoto',
                   'fp_gaussian', 'fp_random_forest']

estimator_name_map = {'mpnn_mve': 'MPNN MVE',
                      'mpnn_gaussian': 'MPNN GP',
                      'mpnn_random_forest': 'MPNN RF',
                      'mpnn_ensemble': 'MPNN Ensemble',
                      'mpnn_tanimoto': 'MPNN Tanimoto Distance',
                      'mpnn_latent_space': 'MPNN Latent Distance',
                      'mpnn_bootstrap': 'MPNN Bootstrap',
                      'mpnn_snapshot': 'MPNN Snapshot Ensembling',
                      'mpnn_dropout10': 'MPNN Dropout (10%)',
                      'mpnn_dropout20': 'MPNN Dropout (20%)',
                      'ffn_mve': 'FFN MVE',
                      'ffn_gaussian': 'FFN GP',
                      'ffn_random_forest': 'FFN RF',
                      'ffn_ensemble': 'FFN Ensemble',
                      'ffn_tanimoto': 'FFN Tanimoto Distance',
                      'ffn_latent_space': 'FFN Latent Distance',
                      'ffn_bootstrap': 'FFN Bootstrap',
                      'ffn_snapshot': 'FFN Snapshot Ensembling',
                      'ffn_dropout10': 'FFN Dropout (10%)',
                      'ffn_dropout20': 'FFN Dropout (20%)',
                      'fp_random_forest': 'FP RF',
                      'fp_gaussian': 'FP GP'}

dataset_name_map = {
    'delaney': 'Delaney',
    'freesolv': 'freesolv',
    'lipo': 'lipo',
    'qm7': 'QM7',
    'logp': 'CLogP'}

proper_estimator_names = [estimator_name_map[estimator_type] for estimator_type in estimator_types]
proper_dataset_names = [dataset_name_map[dataset] for dataset in data_sources]

split_name_map = {'random': 'Random Split',
                  'scaffold': 'Scaffold Split'}

evaluations_df = pd.DataFrame(columns = ['Estimator',
                                         'Data Set',
                                         'Task',
                                         'Split',
                                         'Spearman\'s Coefficient',
                                         'NLL',
                                         'Average NLL',
                                         'Calibrated NLL',
                                         'Average Calibrated NLL',
                                         'Calibration Slope',
                                         'Calibration Intercept',
                                         'Optimal NLL',
                                         'Average Optimal NLL',
                                         'Miscalibration Area',
                                         'File Path'])

### Evaluate Uncalibrated Uncertainty Estimates

In [84]:
for estimator in estimator_types:
    for data_source in data_sources:
        for split in ['random', 'scaffold']:
            uncalibrated_path = f'../uncertainty_evaluation/uncalibrated/{estimator}/{data_source}/{split}/*.txt'
            uncalibrated_files = glob.glob(uncalibrated_path)
            for uncalibrated_file in uncalibrated_files:
                all_evaluations = UncertaintyEvaluator.evaluate(uncalibrated_file, evaluation_metrics)
                core_path = uncalibrated_file[39:]
                for task, task_evaluations in all_evaluations.items():
                    rho = task_evaluations['spearman']['rho']
                    nll = -1 * task_evaluations['log_likelihood']['log_likelihood']
                    average_nll = -1 * task_evaluations['log_likelihood']['average_log_likelihood']
                    optimal_nll = -1 * task_evaluations['log_likelihood']['optimal_log_likelihood']
                    average_optimal_nll = -1 * task_evaluations['log_likelihood']['average_optimal_log_likelihood']
                    miscalibration_area = task_evaluations['calibration_auc']['miscalibration_area']
                    evaluations_df = evaluations_df.append(
                        {'Estimator': estimator_name_map[estimator],
                        'Data Set': dataset_name_map[data_source],
                        'Task': task,
                        'Split': split_name_map[split],
                        'Spearman\'s Coefficient': rho,
                        'NLL': nll,
                        'Average NLL': average_nll,
                        'Calibrated NLL': 0,
                        'Average Calibrated NLL': 0,
                        'Optimal NLL': optimal_nll,
                        'Average Optimal NLL': average_optimal_nll,
                        'Miscalibration Area': miscalibration_area,
                        'File Path': core_path}, ignore_index=True)

In [85]:
labels = list('abcdefghijklmnopqrstuvwxyz')
annotations = list()
for i, label in enumerate(labels):
    annotations.append(dict(text=label,
                            x=0,
                            y=(5-i)/5,
                            xref='paper',
                            yref='paper',
                            showarrow=False,
                            textangle=0))

def update_label(a):
    dataset = a.text.split('=')[1]
    label = labels[proper_dataset_names.index(dataset)]
    return a.update(text=f'<b>({label})</b> {dataset}')

def update_estimator_label(a):
    estimator = a.text.split('=')[1]
    label = labels[proper_estimator_names.index(estimator)]
    return a.update(text=f'<b>({label})</b> {estimator}')

def update_metric_label(a):
    dataset = a.text.split('=')[1]
    label = labels[['Spearman\'s Coefficient', 'Miscalibration Area', 'Average NLL'].index(dataset)]
    return a.update(font=dict(size=20), text=f'<b>({label})</b> {dataset}')

def format_plot(fig):
    fig.update_layout(legend=dict(xanchor='center', x=0.5, y=-0.25, orientation='h', title=None),
                  margin=go.layout.Margin(
                    l=50,
                    r=50,
                    b=0,
                    t=10,
                    pad=0),
                  autosize=False,
                  font=dict(
                      size=24),
                 )

    fig.for_each_annotation(update_label)

### Calibrate Uncertainty Estimates and Reevaluate

In [86]:
for estimator in estimator_types:
    for data_source in data_sources:
        for split in ['random', 'scaffold']:
            path = f'../uncertainty_evaluation/uncalibrated/{estimator}/{data_source}/{split}/*.txt'
            files = glob.glob(path)
            
            for file in files:
                new_path = '../uncertainty_evaluation/calibrated/' + file[39:]

                calibrated_log, coefficients = UncertaintyEvaluator.calibrate([lambda x: x, lambda x: 1], [1, 0], file)
                
                if not os.path.exists(os.path.dirname(new_path)):
                    os.makedirs(os.path.dirname(new_path))
                f = open(new_path, 'w+')
                json.dump(calibrated_log, f)
                f.close()
                
                all_evaluations = UncertaintyEvaluator.evaluate(new_path, evaluation_metrics)
                for task, task_evaluations in all_evaluations.items():
                    index = evaluations_df[evaluations_df['Task'] == task][evaluations_df['File Path'] == new_path[37:]].index[0]
                    calibrated_nll = -1 * task_evaluations['log_likelihood']['log_likelihood']
                    average_calibrated_nll = -1 * task_evaluations['log_likelihood']['average_log_likelihood']
                    evaluations_df.at[index, 'Calibrated NLL'] = calibrated_nll
                    evaluations_df.at[index, 'Average Calibrated NLL'] = average_calibrated_nll
                    evaluations_df.at[index, 'Calibration Slope'] = coefficients[task][0]
                    evaluations_df.at[index, 'Calibration Intercept'] = coefficients[task][1]

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: U

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: U

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: U

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: U

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: U

...

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: U

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: U

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: U

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: U

 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/scipy/optimize/optimize.py:1058: RuntimeWarning:divide by zero encountered in double_scalars
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning:Boolean Series key will be reindexed to match DataFrame index.
 /home/liorh/miniconda3/envs/chemprop_uncertainty/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarni

### Compute Supplemental Evaluations

In [87]:
evaluations_df['Average NLL Difference'] = evaluations_df['Average NLL'] - evaluations_df['Average Optimal NLL']
evaluations_df['Average Calibrated NLL Difference'] = evaluations_df['Average Calibrated NLL'] - evaluations_df['Average Optimal NLL']

### Sort the Data

In [88]:
evaluations_df['Estimator Order'] = evaluations_df['Estimator'].map({
    key: proper_estimator_names.index(key) for key in proper_estimator_names
})

evaluations_df['Data Set Order'] = evaluations_df['Data Set'].map({
    key: proper_dataset_names.index(key) for key in proper_dataset_names
})

In [89]:
evaluations_df = evaluations_df.sort_values(by=['Data Set Order', 'Estimator Order'])

### Save the Data

In [112]:
evaluations_df.to_csv('evaluations.csv')

### Construct a Second Dataframe with Only those Methods that Produce Confidence Bounds

In [90]:
uncalibrated_uq_methods = ['MPNN Tanimoto Distance',
                           'MPNN Latent Distance',
                           'FFN Tanimoto Distance',
                           'FFN Latent Distance']

In [91]:
precalibrated_evaluations_df = evaluations_df.copy()

for method in uncalibrated_uq_methods:
    precalibrated_evaluations_df.NLL *= (evaluations_df['Estimator'] != method)
    precalibrated_evaluations_df['Average NLL'] *= (evaluations_df['Estimator'] != method)
    precalibrated_evaluations_df['Average NLL Difference'] *= (evaluations_df['Estimator'] != method)
    precalibrated_evaluations_df['Miscalibration Area'] *= (evaluations_df['Estimator'] != method)

precalibrated_evaluations_df.NLL.replace(to_replace=[0], value=np.nan, inplace=True)
precalibrated_evaluations_df['Average NLL'].replace(to_replace=[0], value=np.nan, inplace=True)
precalibrated_evaluations_df['Average NLL Difference'].replace(to_replace=[0], value=np.nan, inplace=True)
precalibrated_evaluations_df['Miscalibration Area'].replace(to_replace=[0], value=np.nan, inplace=True)

# Visualizations

### Spearman's

In [93]:
fig = px.box(evaluations_df,
             x='Estimator',
             y="Spearman's Coefficient",
             color='Split',
             facet_row='Data Set',
             points=False,
             height=468*4,
             width=234*4,
             color_discrete_sequence=['#f49a94', '#50c2f2'],
             range_y=[-0.35, 0.8], labels={"Spearman's Coefficient": '\u03C1'})

format_plot(fig)

fig.update_xaxes(
    ticks='outside',
    tickson='boundaries',
    title=None
)

fig.update_yaxes( 
        tickmode = 'linear',
        tick0 = 0.0,
        dtick = 0.25)

fig.show()

### Miscalibration Area

In [94]:
fig = px.box(precalibrated_evaluations_df,
             x='Estimator',
             y='Miscalibration Area',
             color='Split',
             facet_row='Data Set',
             points=False,
             height=468*4,
             width=234*4,
             color_discrete_sequence=['#f49a94', '#50c2f2'],
             range_y=[0, 0.5], labels={'Miscalibration Area': 'Area'})

format_plot(fig)

fig.update_xaxes(
    ticks='outside',
    tickson='boundaries',
    title=None
)

fig.update_yaxes( 
        tickmode = 'linear',
        tick0 = 0.0,
        dtick = 0.125)

fig.show()

### Nll

##### Without FFN MVE

In [95]:
select_precalibrated_evaluations_df = precalibrated_evaluations_df.copy()

select_precalibrated_evaluations_df['Average NLL'] *= (evaluations_df['Estimator'] != 'FFN MVE')
select_precalibrated_evaluations_df['Average NLL'].replace(to_replace=[0], value=np.nan, inplace=True)

In [96]:
fig = px.box(select_precalibrated_evaluations_df,
             x='Estimator',
             y='Average NLL',
             color='Split',
             facet_row='Data Set',
             points=False,
             height=468*4,
             width=234*4,
             color_discrete_sequence=['#f49a94', '#50c2f2'],
            labels={'Average NLL': 'NLL'})

fig.update_xaxes(
    ticks='outside',
    tickson='boundaries',
    title=None
)

fig.update_yaxes(matches=None)

format_plot(fig)

fig.show()

##### With FFN MVE

In [97]:
fig = px.box(precalibrated_evaluations_df,
             x='Estimator',
             y='Average NLL',
             color='Split',
             facet_row='Data Set',
             points=False,
             height=468*4,
             width=234*4,
             color_discrete_sequence=['#f49a94', '#50c2f2'],
            labels={'Average NLL': 'NLL'})

fig.update_xaxes(
    ticks='outside',
    tickson='boundaries',
    title=None
)

fig.update_yaxes(matches=None)

format_plot(fig)

fig.show()

### Calibrated NLL

##### Without MPNN MVE

In [98]:
select_evaluations_df = evaluations_df.copy()

select_evaluations_df['Average Calibrated NLL'] *= (evaluations_df['Estimator'] != 'MPNN MVE')
select_evaluations_df['Average Calibrated NLL'].replace(to_replace=[0], value=np.nan, inplace=True)

In [99]:
fig = px.box(select_evaluations_df,
             x='Estimator',
             y='Average Calibrated NLL',
             color='Split',
             facet_row='Data Set',
             points=False,
             height=468*4,
             width=234*4,
             color_discrete_sequence=['#f49a94', '#50c2f2'],
             labels={'Average Calibrated NLL': 'cNLL'})

fig.update_xaxes(
    ticks='outside',
    tickson='boundaries',
    title=None
)

fig.update_yaxes(matches=None)

format_plot(fig)

fig.show()

##### With MPNN MVE

In [100]:
fig = px.box(select_evaluations_df,
             x='Estimator',
             y='Average Calibrated NLL',
             color='Split',
             facet_row='Data Set',
             points=False,
             height=468*4,
             width=234*4,
             color_discrete_sequence=['#f49a94', '#50c2f2'],
             labels={'Average Calibrated NLL': 'cNLL'})

fig.update_xaxes(
    ticks='outside',
    tickson='boundaries',
    title=None
)

fig.update_yaxes(matches=None)


format_plot(fig)

fig.show()

## Calibration Coefficients

In [101]:
evaluations_df['Capped Calibration Slope'] = evaluations_df['Calibration Slope'].apply(lambda x: 10 if x > 10 else (0 if x < 0 else x))

In [102]:
fig = px.histogram(evaluations_df,
             x='Capped Calibration Slope',
             color='Split',
             facet_col='Estimator',
                   facet_col_wrap=4,
             height=468*2,
             width=234*4,
            nbins=20,
             color_discrete_sequence=['#f49a94', '#50c2f2'],
             labels={'Calibrated NLL': 'cNLL'})

fig.for_each_annotation(update_estimator_label)

fig.show()

## NLL Differences

In [103]:
fig = px.box(precalibrated_evaluations_df,
             x='Estimator',
             y='Average NLL Difference',
             color='Split',
             facet_row='Data Set',
             points=False,
             height=468*4,
             width=234*4,
             color_discrete_sequence=['#f49a94', '#50c2f2'],
            labels={'Average NLL Difference': 'NLL Difference'})

fig.update_xaxes(
    ticks='outside',
    tickson='boundaries',
    title=None
)

fig.update_yaxes(matches=None)

format_plot(fig)

fig.show()

In [104]:
fig = px.box(precalibrated_evaluations_df,
             x='Estimator',
             y='Average Calibrated NLL Difference',
             color='Split',
             facet_row='Data Set',
             points=False,
             height=468*4,
             width=234*4,
             color_discrete_sequence=['#f49a94', '#50c2f2'],
            labels={'Average Calibrated NLL Difference': 'cNLL Difference'})

fig.update_xaxes(
    ticks='outside',
    tickson='boundaries',
    title=None
)

fig.update_yaxes(matches=None)

format_plot(fig)

fig.show()

## RMSE

In [105]:
rmse_df = pd.DataFrame(columns = ['Estimator',
                                  'Data Set',
                                  'Selection',
                                  'RMSE'])

estimators = ['mpnn_ensemble', 'mpnn_mve', 'mpnn_tanimoto', 'mpnn_random_forest',
              'ffn_ensemble', 'ffn_mve', 'ffn_tanimoto', 'ffn_random_forest']
split = 'random'

for estimator in estimators:
    for data_source in data_sources:
        uncalibrated_path = f'../uncertainty_evaluation/uncalibrated/{estimator}/{data_source}/{split}/*.txt'
        uncalibrated_files = glob.glob(uncalibrated_path)

        percents = [100, 50, 25, 10, 5]
        avg_rmse = {percent: 0 for percent in percents}
        for uncalibrated_file in uncalibrated_files:
            f = open(uncalibrated_file)
            test_log = json.load(f)['test']

            for task, task_info in test_log.items():
                for percent in percents:
                    sets_by_uncertainty = task_info['sets_by_uncertainty']
                    
                    topx = int(len(sets_by_uncertainty) * percent / 100)

                    mse = 0

                    for set_ in sets_by_uncertainty[-topx:]:
                        mse += set_['error']**2 / topx

                    avg_rmse[percent] += np.sqrt(mse) / len(uncalibrated_files) 
            f.close()
        for percent in percents:
            rmse_df = top5_df.append({'Estimator': estimator_name_map[estimator],
                            'Data Set': dataset_name_map[data_source],
                            'Selection': f'Top {percent}%',
                            'RMSE': avg_rmse[percent]}, ignore_index=True)

In [106]:
fig = px.bar(rmse_df,
             x='Estimator',
             y='RMSE',
             color='Selection',
             height=468*4,
             width=234*4,
             facet_row='Data Set')

# Change the bar mode

fig.update_yaxes(matches=None)
fig.update_layout(barmode='group')

format_plot(fig)


fig.show()

### Wilcoxon

In [107]:
def wilcoxon(estimator1, estimator2, metric, average = False, reverse = False):
    estimator1_df = evaluations_df[(evaluations_df['Estimator'] == estimator1) & (evaluations_df['Split'] == 'Random Split')]
    estimator2_df = evaluations_df[(evaluations_df['Estimator'] == estimator2) & (evaluations_df['Split'] == 'Random Split')]

    # Perform average of random splits for each dataset.
    if average:
        estimator1_metrics = []
        estimator2_metrics = []
        for dataset in proper_dataset_names:
            estimator1_metrics.append(estimator1_df[estimator1_df['Data Set'] == dataset][metric].median())
            estimator2_metrics.append(estimator2_df[estimator2_df['Data Set'] == dataset][metric].median())
    else:
        estimator1_metrics = estimator1_df.sort_values(['Data Set', 'File Path'])[metric].to_list()
        estimator2_metrics = estimator2_df.sort_values(['Data Set', 'File Path'])[metric].to_list()

    differences = sorted([x1 - x2 for x1, x2 in zip(estimator1_metrics, estimator2_metrics)], key = lambda x: abs(x))

    R_plus = 0
    R_minus = 0
    for i in range(len(differences)):
        if differences[i] > 0:
            R_plus += i + 1
        elif differences[i] == 0:
            R_plus += (i + 1) / 2
            R_minus += (i + 1) / 2
        else:
            R_minus += i + 1

    T = R_plus
    if reverse:
        T = R_minus
    N = len(differences)
    return (T - N * (N+1)/4)/(np.sqrt(N*(N+1)*(2*N+1)/24))

#### Every Random Split

In [108]:
wilcoxon_split_df = pd.DataFrame(columns = ['Primary Estimator',
                                            'Secondary Estimator',
                                            'Metric',
                                            'Z-Score'])

metrics = {'Spearman\'s Coefficient': False,
           'Miscalibration Area': True,
           'Average NLL': True}

for primary_estimator in reversed(proper_estimator_names):
    for secondary_estimator in proper_estimator_names:
        for metric, reverse in metrics.items():
            wilcoxon_split_df = wilcoxon_split_df.append({'Primary Estimator': primary_estimator,
                                    'Secondary Estimator': secondary_estimator,
                                    'Metric': metric,
                                    'Z-Score': wilcoxon(primary_estimator,
                                                        secondary_estimator,
                                                        metric,
                                                        reverse=reverse)}, ignore_index=True)

In [109]:
fig = px.density_heatmap(wilcoxon_split_df,
                 x='Secondary Estimator',
                 y='Primary Estimator',
                 z='Z-Score',
                 height=468*1.8,
                 width=234*8,
                 facet_col='Metric',
                 histfunc='avg',
                 color_continuous_scale=['#8845AD','#fff', '#49913A'],
                 )

fig.update_layout(coloraxis=dict(colorbar=dict(title='Z-Score', thickness=20, xpad=0, ypad=0)),
                  autosize=False,
                  xaxis = dict(tickfont = dict(size=17)), 
                  yaxis = dict(tickfont = dict(size = 17)),
                  font=dict(size=17))

def update_metric_label(a):
    dataset = a.text.split('=')[1]
    label = labels[['Spearman\'s Coefficient', 'Miscalibration Area', 'Average NLL'].index(dataset)]
    return a.update(font=dict(size=20), text=f'<b>({label})</b> {dataset}')

fig.for_each_annotation(update_metric_label)

fig.show()

##### Averages

In [110]:
wilcoxon_split_df = pd.DataFrame(columns = ['Primary Estimator',
                                            'Secondary Estimator',
                                            'Metric',
                                            'Z-Score'])

metrics = {'Spearman\'s Coefficient': False,
           'Miscalibration Area': True,
           'Average NLL': True}

for primary_estimator in reversed(proper_estimator_names):
    for secondary_estimator in proper_estimator_names:
        for metric, reverse in metrics.items():
            wilcoxon_split_df = wilcoxon_split_df.append({'Primary Estimator': primary_estimator,
                                    'Secondary Estimator': secondary_estimator,
                                    'Metric': metric,
                                    'Z-Score': wilcoxon(primary_estimator,
                                                        secondary_estimator,
                                                        metric,
                                                        reverse=reverse,
                                                        average=True)}, ignore_index=True)

In [111]:
fig = px.density_heatmap(wilcoxon_split_df,
                 x='Secondary Estimator',
                 y='Primary Estimator',
                 z='Z-Score',
                 height=468*1.8,
                 width=234*8,
                 facet_col='Metric',
                 histfunc='avg',
                 color_continuous_scale=['#8845AD','#fff', '#49913A'],
                 )

fig.update_layout(coloraxis=dict(colorbar=dict(title='Z-Score', thickness=20, xpad=0, ypad=0)),
                  autosize=False,
                  xaxis = dict(tickfont = dict(size=17)), 
                  yaxis = dict(tickfont = dict(size = 17)),
                  font=dict(size=17))

fig.for_each_annotation(update_metric_label)

fig.show()